In [1]:
import pandas as pd
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt
import konlpy
import re
from tqdm import tqdm
from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim import corpora
from gensim.models.callbacks import PerplexityMetric
import logging
import pyLDAvis.gensim_models
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis.sklearn # sklearn의 ldamodel에 최적화된 라이브러리

In [2]:
df = pd.read_csv("C:\\bigdata\\티셔츠_1.csv")
df = df.drop_duplicates(['상품후기'], ignore_index = True)
df

,Unnamed: 0,상품후기,별점
0,0,"디자인, 색상, 배송, 재질 모두 만족합니다. 추천합니다.",5
1,2,가성비 좋개 장산거 같아요 다음에도 이용할거 같네요 ~~,5
2,3,색상이 찐하고 어두운블루라 부담없이 입기좋은것 같습니다 핏도 여유있어 편하네요,5
3,4,옷 질감고 부들부들하고 사이즈도 적당히 세미오버핏이에요,5
4,5,다크 그레이 색 반팔필요해서 샀는데 역시이쁘네요,5
...,...,...,...
23519,2045,그냥 이 가격에 이퀄이면 두말말고 그냥 지르시면됩니다.,5
23520,2046,탄탄하고 핏도 딱 좋아요 목은 빨아봐야 알겠지만 일년은 무난히 입을수 있을정도로 탄...,5
23521,2047,옷 자체의 발색도 괜찮고 질도\n나쁘지 않은 것 같습니다.,4
23522,2048,색상도 쨍하니 발색이 좋고 질도\n나쁘지 않은 것 같습니다.,4


In [3]:
#공백과 한글 제거
df['상품후기'] = df['상품후기'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df

C:\Users\km022\AppData\Local\Temp/ipykernel_38948/1783146766.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['상품후기'] = df['상품후기'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


,Unnamed: 0,상품후기,별점
0,0,디자인 색상 배송 재질 모두 만족합니다 추천합니다,5
1,2,가성비 좋개 장산거 같아요 다음에도 이용할거 같네요,5
2,3,색상이 찐하고 어두운블루라 부담없이 입기좋은것 같습니다 핏도 여유있어 편하네요,5
3,4,옷 질감고 부들부들하고 사이즈도 적당히 세미오버핏이에요,5
4,5,다크 그레이 색 반팔필요해서 샀는데 역시이쁘네요,5
...,...,...,...
23519,2045,그냥 이 가격에 이퀄이면 두말말고 그냥 지르시면됩니다,5
23520,2046,탄탄하고 핏도 딱 좋아요 목은 빨아봐야 알겠지만 일년은 무난히 입을수 있을정도로 탄...,5
23521,2047,옷 자체의 발색도 괜찮고 질도나쁘지 않은 것 같습니다,4
23522,2048,색상도 쨍하니 발색이 좋고 질도나쁘지 않은 것 같습니다,4


In [4]:
df[df['별점']==3].sample(10) #3점 

,Unnamed: 0,상품후기,별점
7081,3802,원단이 탄탄하고 비침이 없어서 편하게 입고다니고 있어요,3
9909,2979,기본티 입니다 부드럽고 편하게 입을 수 있습니다,3
18866,869,사이즈넉넉하게 잘 맞고 생각보다 목이 좀 파였어요원래 반팔보다 쇄골이 더 나온...,3
14036,3594,딱 맞네요 가격도 괜찮고 편하게입고다니기에 좋네요,3
13199,2675,목부분이 잘늘어나요 원짜리에 무얼 바라겠어여,3
13956,3510,엄청 이쁘고 그런건없고 그냥 기본템 무난무난합니다,3
9461,2504,그냥 면 반팔입니다 레이어드용으로 사서 아무 기대 안했는데 그냥 반팔이에요 근데 레...,3
5706,2265,언제나 한사이즈 작게 나오는 느낌의 무탠아니다 싶어도 가격 때문에 한번 더 믿어보게...,3
8733,1716,무신사 로고가 뒷 목 부분으로 비치지만 그거 제외하면 아주 좋습니다,3
17184,3069,생각 보다 너무 큰 것같아요 앞으로는 한치수 작은거 사려구요,3


In [5]:
df[df['별점']==4].sample(10) #4점

,Unnamed: 0,상품후기,별점
10897,60,너무 맘에들어요 여름에 정말 많이입고 다녔습니다,4
21809,139,무신사 스탠다드 릴렉스핏 반팔티 머스타드 색감 나쁘지 않네요좋습니다,4
19664,1756,목부분도 좀 큰느낌이고 대체적으로 좀 크게 나온거 같습니다뭔가 목만 좀 덜 크게 나...,4
20354,2514,무탠다드 티셔츠 답게 모자랄것 없이 딱 무난하네요,4
7485,270,제 사이즈랑 맞아서 정말 좋네요 나중에 또사려구요,4
11846,1168,편하고 무지라 자주입어요 기본템 ㅇㅈ네이비 별로 안좋아해도 사두변 입을일 분명히 있음,4
6403,3050,가격대비 무난해서 좋은 것 같아요 몇 벌 더 사야겟어요,4
13137,2611,막 입기 편하고 좋습니다 무난무난합니다 편합니다,4
11883,1209,면 질감 괜찮은 오버핏 반팔티셔츠이고 머쉬룸 색상 색감이 꽤 예쁩니다 평소 입는 반...,4
14465,79,너무 크지 않고 딱 맞아서 좋아여 ㅎㅎ 목부분 짱짱해서 건조기 돌려도 괜찮네여 ㅎㅎ...,4


In [6]:
a = [0 if score in range(1,3) else 1 for score in df['별점']]
df['pn']=a

df

,Unnamed: 0,상품후기,별점,pn
0,0,디자인 색상 배송 재질 모두 만족합니다 추천합니다,5,1
1,2,가성비 좋개 장산거 같아요 다음에도 이용할거 같네요,5,1
2,3,색상이 찐하고 어두운블루라 부담없이 입기좋은것 같습니다 핏도 여유있어 편하네요,5,1
3,4,옷 질감고 부들부들하고 사이즈도 적당히 세미오버핏이에요,5,1
4,5,다크 그레이 색 반팔필요해서 샀는데 역시이쁘네요,5,1
...,...,...,...,...
23519,2045,그냥 이 가격에 이퀄이면 두말말고 그냥 지르시면됩니다,5,1
23520,2046,탄탄하고 핏도 딱 좋아요 목은 빨아봐야 알겠지만 일년은 무난히 입을수 있을정도로 탄...,5,1
23521,2047,옷 자체의 발색도 괜찮고 질도나쁘지 않은 것 같습니다,4,1
23522,2048,색상도 쨍하니 발색이 좋고 질도나쁘지 않은 것 같습니다,4,1


In [7]:
df['pn'].value_counts()

1    23398
0      126
Name: pn, dtype: int64

In [8]:
df.별점.unique()

array([5, 4, 3, 1, 2], dtype=int64)

In [9]:
df[df['pn']==0].sample(10)   # 부정인 리뷰들

,Unnamed: 0,상품후기,별점,pn
5208,1708,걍 내의 같아요기대를 넘 많이했나 평균보다 늘어나 제작된 넥라인이 특히 그렇게 만듭...,1,0
10174,3260,저번에삿을때는 여유로운 핏 이번이 산건 작은느낌 똑같은 옷인대 뭐가바뀐건가,2,0
18090,40,무신사중에서 별로 비추하거싶은 항목 입니다 그냥 맘에 안끄리면 사지마세용,2,0
11207,412,예전에 한번 샀다가 괜찮아서 그대로 다시 구입했는데 많이 바뀌었네요사이즈더커짐 두께...,2,0
9262,2289,상품은 평소 입는거라 만족하지만 배송이 늦었어요,2,0
21380,3660,솔직히 단품 패션아이템으론별로에요티셔츠 질 자체는 괜찮아보이는데핏이 그냥 별로에요 ...,2,0
8671,1651,가격대비 무난하네요다만 화이트 블랙 두색상 시켰는데동일한 제품임에도 입었을 때 사이...,1,0
7524,311,처음 받고 입어볼 때까지는 완전 대만족했어요근데 세탁후에 싸그리 사라지네요 이거 옷...,1,0
23435,1948,딱 만원짜리 티셔츠입니다사이즈 생각보다 크고 실밥 마감도 별로네요,1,0
18357,319,이 가격에 얇을 거란 생각은 했지만실크마냥 얇네요 젖꼭지 다 보임별로임,2,0


In [10]:
n = df[df['pn']==0]
n

,Unnamed: 0,상품후기,별점,pn
54,58,상품 받아보고 내가알던 무탠다드 맞나 싶어서 올리는 포토후기최근엔 귀찮아서 포토후기...,1,0
549,593,목 마감이 상당히 불량합니다 같이산 그레이티셔츠는 마감이 깨끗한데 이제품은 사진에서...,1,0
759,822,저번주에 받아서 오늘 처음 입는데 무슨 목라인이 무슨 사람 얼굴 개는 들어갈 정도로...,1,0
906,986,같은 무신사 옷 두개 샀는데 옷 부분 마감이 다르더라고요 만든 곳이 다른가봅니다 목...,2,0
1293,1395,작년것 생각하고 주문했는데 사이즈가 완전히 달라졌네요 환불하기 귀찮아서 그냥 잠옷으...,1,0
...,...,...,...,...
22938,1376,사진에서 보시는것처럼 어깨쪽이 울어잇어요 팔을 들었을때 같이 딸려서 올라가서어깨쪽이...,2,0
22964,1406,국방색을 떠올리기하는 카키입니다 물론 자주입어서 색이바란느낌전체적으로는 가격만큼의 ...,2,0
23247,1731,사이즈 루즈핏으로 잘 맞음 근데 목 뒤 무신사 택 바느질 진짜 폭망 저질 퀄리티...,1,0
23435,1948,딱 만원짜리 티셔츠입니다사이즈 생각보다 크고 실밥 마감도 별로네요,1,0


In [11]:
p = df[df['pn']==1] # 긍정적 리뷰들
p

,Unnamed: 0,상품후기,별점,pn
0,0,디자인 색상 배송 재질 모두 만족합니다 추천합니다,5,1
1,2,가성비 좋개 장산거 같아요 다음에도 이용할거 같네요,5,1
2,3,색상이 찐하고 어두운블루라 부담없이 입기좋은것 같습니다 핏도 여유있어 편하네요,5,1
3,4,옷 질감고 부들부들하고 사이즈도 적당히 세미오버핏이에요,5,1
4,5,다크 그레이 색 반팔필요해서 샀는데 역시이쁘네요,5,1
...,...,...,...,...
23519,2045,그냥 이 가격에 이퀄이면 두말말고 그냥 지르시면됩니다,5,1
23520,2046,탄탄하고 핏도 딱 좋아요 목은 빨아봐야 알겠지만 일년은 무난히 입을수 있을정도로 탄...,5,1
23521,2047,옷 자체의 발색도 괜찮고 질도나쁘지 않은 것 같습니다,4,1
23522,2048,색상도 쨍하니 발색이 좋고 질도나쁘지 않은 것 같습니다,4,1


## negative 토픽모델링

In [12]:
# 특수 문자 제거
n['clean'] = n['상품후기'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
# 길이가 2이하인 단어는 제거 (길이가 짧은 단어 제거)
n['clean'] = n['clean'].apply(lambda x: ' '.join([w for w in str(x).split() if len(w)>2]))

C:\Users\km022\AppData\Local\Temp/ipykernel_38948/2978971600.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  n['clean'] = n['상품후기'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
C:\Users\km022\AppData\Local\Temp/ipykernel_38948/2978971600.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  n['clean'] = n['상품후기'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
C:\Users\km022\AppData\Local\Temp/ipykernel_38948/2978971600.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  n['clean'] = 

### 토큰화


In [13]:
def tokenize_korean_text(text):
    text = re.sub(r'[^,.?!\w\s]','', text)  ## ,.?!와 문자+숫자+_(\w)와 공백(\s)만 남김  # 앞에 r을 붙여주면 deprecation warning이 안뜸 (raw string으로 declare)
    
    okt = konlpy.tag.Okt()
    Okt_morphs = okt.pos(text)   # stem=True로 설정하면 동사원형으로 바꿔서 return
    
    words = []
    for word, pos in Okt_morphs:
        if pos == 'Noun':  # 이 경우에는 명사만 남김
            words.append(word)

    words_str = ' '.join(words)
    return words_str

# review_df['Content']를 하나씩 tokenize해서 list로 저장
tokenized_list = []

for text in df['상품후기']:
    tokenized_list.append(tokenize_korean_text(text))

print(len(tokenized_list))
print(tokenized_list)

pd.DataFrame(tokenized_list).to_csv('C:\\bigdata\\review_tokenized.csv')

23524
['디자인 색상 배송 재질 모두 추천', '가성 비 개 장산 거 다음 이용', '색상 어 두운 블루 부담 것 핏 여유', '옷 질감 사이즈 세미 버핏', '다크 그레이 색 팔 역시', '사진 좀 실제 색', '배송 재질 사이즈 음 살짝 느낌', '스탠다드 티셔츠 데 일리', '색깔 생각 아주 조금 재질 사이즈', '동생 려고 어요 생각 거 평소 동생 사이즈 거', '좀 편 한번 세탁 후 건조기 핏', '크게 집 한여름 대신', '굿굿 수수 이브 짱', '샤카 무탠 다드 한장 무탠 다드 착용 감 제일 원단 느낌 넥라인 쪼끔', '색감 완전 화이트 두께 편 거의 품 입 수도 품 약간 티 날 거 자켓 셔츠 안 듯 후드 투맨 사이즈 천만다행 보통 티셔트 라지 큰일 날 뻔 버핏 단품 너 로 가야 듯', '상품 배송 전부 다른 회사 제품', '기본 빵 이상은 듯', '저 바지 크게 세탁 수축 걸 고려 생각 수축 아주', '고민 결과 립 중간 한번 더 봉제 방지 단감 면 마음', '저렴 가격 때문 추천', '사이즈 작 줄 생각 크게 그냥 티', '역시 가성 비 최고 애용', '넥 길이 정말 맘에듭니 사이즈 해', '가성 비 사이즈 버핏 추천', '생각 입 만해 코디 도전 것', '그레이 색상 옷 사이즈 레이어 드용', '역시 기본 티 핏 장난 정말', '기본 티 어디', '무탠다드 정말 옷 항상', '집 디자인 색감', '예전 원단 세탁 후 보풀 전혀', '벌써 장 구매 너 안', '와인 색상 구매 색상 살짝 가격 대비 만족', '좀더 데 일리', '과연 이 컬러 입 수 가격 한번 는걸', '사이즈 널널 색상 하얀색 넥라인', '기장 좀 색감 화면 통 걸', '아주 재질 사서', '생각 어 두운 색상 티', '티 성비 굿굿', '디자인 핏 소재 여러가지 마음 옷', '사이즈 잘못 선택 집 옷 착용 감', '가격 대비 가성 가슴 제의 눈 생각 달이', '어디 티', '가격 핏 전체 무난', '원단 평소 여름 옷 생각 함', '무지 티 색깔 사진 그대', '넥라인 

In [14]:
drop_corpus = []

for index in range(len(tokenized_list)):
    corpus = tokenized_list[index]
    if len(set(corpus.split())) < 3:   # 같은 단어 1-2개만 반복되는 corpus도 지우기 위해 set()을 사용
        df.drop(index, axis='index', inplace=True)
        drop_corpus.append(corpus)
    
for corpus in drop_corpus:
    tokenized_list.remove(corpus)

df.reset_index(drop=True, inplace=True)

print(len(tokenized_list))
print(len(df))

22078
22078


## Vectorization & LDA

### Veoctor화

In [15]:
#LDA 는 Count기반의 Vectorizer만 적용 
count_vectorizer = CountVectorizer(max_df=0.1, max_features=1000, min_df=2, ngram_range=(1,2))
    # 2개의 문서 미만으로 등장하는 단어는 제외, 전체의 10% 이상으로 자주 등장하는 단어는 제외
    # bigram도 포함

feat_vect = count_vectorizer.fit_transform(tokenized_list)
print('CountVectorizer Shape:', feat_vect.shape)

CountVectorizer Shape: (22078, 1000)


In [16]:
lda = LatentDirichletAllocation(n_components=20)  # 토픽 수는 20개로 설정
lda.fit(feat_vect)

LatentDirichletAllocation(n_components=20)

In [17]:
def display_topics(model, feature_names, num_top_words):
    for topic_index, topic in enumerate(model.components_):
        print('Topic #', topic_index+1)

        # components_ array에서 가장 값이 큰 순으로 정렬했을 때, 그 값의 array index를 반환. 
        topic_word_indexes = topic.argsort()[::-1]
        top_indexes=topic_word_indexes[:num_top_words]
        
        # top_indexes대상인 index별로 feature_names에 해당하는 word feature 추출 후 join으로 concat
        feature_concat = ' '.join([feature_names[i] for i in top_indexes])                
        print(feature_concat)

# CountVectorizer객체내의 전체 word들의 명칭을 get_features_names( )를 통해 추출
feature_names = count_vectorizer.get_feature_names()

# Topic별 가장 연관도가 높은 word를 10개만 추출
display_topics(lda, feature_names, 10)

Topic # 1
무난 약간 크게 블랙 무난 무난 치수 화이트 사이즈 크게 그레이 무난 가격
Topic # 2
다드 무탠 무탠 다드 역시 역시 무탠 다드 가성 역시 가성 티셔츠 무텐 무텐 다드
Topic # 3
두께 여름 그냥 목도 그냥 무난 여유 셔츠 비치 한철 포인트
Topic # 4
색도 오버 넥라인 소재 라지 제일 세미 세미 오버 오버 사이즈 다른
Topic # 5
생각 기장 질도 일리 조금 루즈 사이즈 생각 그냥 느낌 소매
Topic # 6
레이어 하나 드용 레이어 드용 이즈 정사 드하 레이어 드하 정사 이즈 항상
Topic # 7
마음 아주 배송 듭니 완전 상품 마음 듭니 굿굿 드네 면도
Topic # 8
대비 가격 대비 자주 퀄리티 추가 이용 추가 구매 사용 정말 대비 퀄리티
Topic # 9
색깔 별로 무지 마감 성비 보리 색깔 별로 요가 요가 성비 별로 구매
Topic # 10
재질 색감 이정 일단 편이 재질 가격 가격 재질 가격 이정 구김 퀄리티
Topic # 11
기본 티셔츠 흰색 구입 매우 검은색 그냥 사고 기본 티셔츠 스탠다드
Topic # 12
최고 제품 가성 최고 컬러 착용 살짝 질감 느낌 심플 두께
Topic # 13
부분 어깨 다음 전체 라인 커서 조금 크기 다만 느낌
Topic # 14
버핏 살짝 생각 다운 활용 추천 느낌 사이즈 다운 생각 사이즈 사이즈 버핏
Topic # 15
진짜 정도 만족 만원 사서 한번 후드 스타일 퀄리티 투맨
Topic # 16
세탁 주문 재질 거의 몇번 벌써 검정 다시 수축 해도
Topic # 17
정말 스탠다드 재질 이번 여름 버핏 걱정 역시 세미 버핏 세미
Topic # 18
추천 사진 길이 색감 그대로 실제 사이즈 구매 부담 두운 참고
Topic # 19
조금 원단 기본 무탠다드 무지 평소 어디 의사 아이템 기본 무지
Topic # 20
색상 다른 품질 색상 구매 크림 다른 색상 네이비 블루 가격 품질 사진


## LDA 시각화

In [18]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, feat_vect, count_vectorizer)
pyLDAvis.display(vis)

C:\Users\km022\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


### 각 문서별로 가장 가까운 topic으로 할당함.

In [19]:
doc_topic = lda.transform(feat_vect)

doc_per_topic_list = []
for n in range(doc_topic.shape[0]):
    topic_most_pr = doc_topic[n].argmax()
    topic_pr = doc_topic[n].max()
    doc_per_topic_list.append([n, topic_most_pr, topic_pr])
    
doc_topic_df = pd.DataFrame(doc_per_topic_list, columns=['Doc_Num', 'Topic', 'Percentage'])

doc_topic_df.head()

,Doc_Num,Topic,Percentage
0,0,19,0.284237
1,1,12,0.350000
2,2,19,0.391934
3,3,16,0.675000
4,4,9,0.610000


In [20]:
doc_topic_df = doc_topic_df.join(df)
doc_topic_df.head()

,Doc_Num,Topic,Percentage,Unnamed: 0,상품후기,별점,pn
0,0,19,0.284237,0,디자인 색상 배송 재질 모두 만족합니다 추천합니다,5,1
1,1,12,0.350000,2,가성비 좋개 장산거 같아요 다음에도 이용할거 같네요,5,1
2,2,19,0.391934,3,색상이 찐하고 어두운블루라 부담없이 입기좋은것 같습니다 핏도 여유있어 편하네요,5,1
3,3,16,0.675000,4,옷 질감고 부들부들하고 사이즈도 적당히 세미오버핏이에요,5,1
4,4,9,0.610000,5,다크 그레이 색 반팔필요해서 샀는데 역시이쁘네요,5,1


In [21]:
doc_topic_df.groupby('Topic')[['Doc_Num']].count() #토픽별로 문서 개수 계산

,Doc_Num
Topic,
0,1469
1,1648
2,1084
3,907
4,1128
5,1266
6,1038
7,954
8,916


In [22]:
for topic in range(len(doc_topic_df['Topic'].unique())):
    print('Topic #', topic+1, '-----------------------------')
    top_pr_topics = doc_topic_df[doc_topic_df['Topic'] == topic].sort_values(by='Percentage', ascending=False)
    print('1. ',top_pr_topics['상품후기'].iloc[0])
    print('2. ',top_pr_topics['상품후기'].iloc[1])
    print('3. ',top_pr_topics['상품후기'].iloc[2])
    print('4. ',top_pr_topics['상품후기'].iloc[3])
    print('5. ',top_pr_topics['상품후기'].iloc[4], '\n') #토픽별로 가장 높은 확률로 할당된 문서 top5 확인

Topic # 1 -----------------------------
1.  널널하게 입는 걸 좋아해서 한치수 크게 샀는데 약간 크게 느껴지네요본인에 맞는 사이즈로 하시면 딱 맞게 입으실 수 있을 것 같아요
2.  회색도 무난한 색이고 핏도 무난하고 전체적으로 무난해요
3.  실속있고 가성비 끝판왕인 것 같아요 핏도 적당하고 무난무난하게 입습니다
4.  크게 입고싶어서 한 사이즈 크게갔는데 예상대로 좋습니다
5.  사이즈는 약간 크게 나왔는데 이 가격이면 닥치고 입겠습니다 

Topic # 2 -----------------------------
1.  역시 무탠ㄷ다드무탠다드 유니클로 보다 갓성비 무탠다드 핏 돟음
2.  역시 가성비갑 무탠다드어중간한거 사는거보다 무탠다드 반팔티 최고
3.  역시 믿고사는 무탠다드기본템은 무탠다드 진리내요
4.  역시 무탠다드 릴렉스 핏 크루넥 반팔 티셔츠네요
5.  역시 무탠다드 믿고가는 무탠다드입니다만족해요 

Topic # 3 -----------------------------
1.  사이즈도 낙낙하고 두께감도 좋아서 여름 내내 잘 입을 것 같습니다
2.  더운 여름에 라임색으로 포인트 주기 좋고 시원시원해보여서 좋아요
3.  이너로 입으려고 샀는데 두께감도있고 비치지도않고 여름에 그냥 가볍게 입고다니기 좋을거같아요 ㅋㅋ
4.  두께도 안비치고 딱이에요여름이라 두께때매 더울수있는데 곧 선선해지면 더 좋죠
5.  사이즈 적당합니다 두께감 약간 있고 비치지않아요 

Topic # 4 -----------------------------
1.  원래 제 키에는 라지를 사야 하지만너무 어벙벙한게 싫어서 미디움을 샀더니세미 오버로 잘 맞습니다원래 너무 릴렉스핏을 싫어해서요팔 긴 것도 싫고
2.  색도 예뻐서 아무데나 입어도 어울릴 것 같아요 다른 색도 구매 예정입니다
3.  가성비 개굿입니다 가볍게 입기좋아요 나중에 다른 색도 구입할 예정입니다
4.    사이즈 세미오버로 좋고 베이지색 바지랑 잘 어울리네요
5.  색도 너무 